In [ ]:
import pandas as pd
#import mysql.connector
import pymysql
import streamlit as st

# MySQL connection setup
db_config = {
    'user': 'root',
    'password': '2594',
    'host': 'localhost',
    'database': 'busdata_db'
}

# Connect to the database
conn = pymysql.connect(**db_config)
cursor = conn.cursor()

# Create the table
create_table_query = """
CREATE TABLE IF NOT EXISTS buses (
    
    route_link TEXT,
    busname VARCHAR(255),
    bustype VARCHAR(255),
    departing_time VARCHAR(255),
    duration VARCHAR(255),
    reaching_time VARCHAR(255),
    star_rating FLOAT,
    price FLOAT,
    seats_available INT
)
"""
cursor.execute(create_table_query)
# Load data from CSV
bus_data = pd.read_csv('APBus_Govt_Private.csv')

# Drop the unnamed first column(there is an unwanted column 0)
bus_data = bus_data.iloc[:, 1:]
# Stripping text from seats_available column
bus_data['seats_available'] = bus_data['seats_available'].str.extract('(\d+)').fillna(0).astype(int)

insert_query = """
    INSERT INTO buses (route_link, busname, bustype, departing_time, duration, reaching_time, star_rating, price, seats_available)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
for index, row in bus_data.iterrows():
    cursor.execute(insert_query, (
        row['route_link'], row['busname'], row['bustype'], row['departing_time'], 
        row['duration'], row['reaching_time'], row['star_rating'], row['price'], 
        row['seats_available']
    ))


conn.commit()

# Close the connection
cursor.close()
conn.close()